In [118]:
import numpy as np
from random import random
from scipy.optimize import minimize

from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

# For H1

In [119]:
def hamiltonian_operator(a, b):
    """
    Creates a*I + b*Z pauli sum 
    that will be our Hamiltonian operator.
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [120]:
a, b = -0.218291, 0.218291
H = hamiltonian_operator(a, b)

In [121]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -0.436582


In [122]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    q = circuit.qregs[0]
    circuit.rx(parameters[0], q[0])
    circuit.ry(parameters[1], q[0])


    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "Z"')

    return circuit


In [123]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "Z"')
    
    shots = 1024
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [124]:
def pauli_operator_to_dict(pauli_operator):
    """
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [125]:
def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = pauli_dict['I'] * quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['Z'] * quantum_module(parameters, 'Z')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z 
    return classical_adder

In [126]:
parameters_array = np.array([np.pi, np.pi]) #initals guess
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="COBYLA", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.436582
The estimated ground state energy from VQE algorithm is: -0.435302951171875


# For H2

In [127]:
def hamiltonian_operator(a, b, c, d, e):
    """
    Creates a*II + b*IZ + c*ZI + d*XX + e*YY pauli sum
    that will be our Hamiltonian operator.
   
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "II"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "ZI"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "XX"},
                   {"coeff": {"imag": 0.0, "real": e}, "label": "YY"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [128]:
a, b, c, d, e = 5.906709 , 0.218291, -6.125 ,-2.143304 ,-2.143304
H = hamiltonian_operator(a, b, c, d, e)

In [129]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -1.7491612220155843


In [130]:
def vqe_circuit(parameters, measure):
    """Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'IZ' stands for Z0 measurement,  'XX' stands for X0X1 measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    circuit = QuantumCircuit(q, c)

    circuit.x(0)
    circuit.ry(parameters[0],q[1])
    circuit.cx(q[1], q[0])
    
    
    
    # measurement
    if measure == 'IZ':
        circuit.barrier(q[0],q[1])
        circuit.measure(q[0], c[0])
    elif measure == 'ZI':
        circuit.barrier(q[0],q[1])
        circuit.measure(q[1], c[1])
    elif measure == 'XX':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi, q[0])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'YY':
        circuit.barrier(q[0],q[1])
        circuit.u(np.pi/2, 0, np.pi/2, q[0])
        circuit.u(np.pi/2,0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    else:
        raise ValueError('Not valid input for measurement: input should be "IZ" or "ZI" or "XX" or "YY"')
    
    return circuit


In [131]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'II':
        return 1
    elif measure == 'IZ':
        circuit = vqe_circuit(parameters, 'IZ')
    elif measure == 'ZI':
        circuit = vqe_circuit(parameters, 'ZI')
    elif measure == 'XX':
        circuit = vqe_circuit(parameters, 'XX')
    elif measure == 'YY':
        circuit = vqe_circuit(parameters, 'YY')
    else:
        raise ValueError('Not valid input for measurement: input should be "IZ" or "ZI" or "XX" or "YY"')
   
    shots = 10000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = -1
        if (measure_result == '00') or (measure_result == '11'):
             sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value



In [132]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {II: 5.906709 , IZ: 0.218291, ZI: -6.125, XX: -2.143304, YY: -2.143304}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [133]:
def vqe(parameters):
       
    # quantum_modules
    quantum_module_II = pauli_dict['II'] * quantum_module(parameters, 'II')
    quantum_module_IZ = pauli_dict['IZ'] * quantum_module(parameters, 'IZ')
    quantum_module_ZI = pauli_dict['ZI'] * quantum_module(parameters, 'ZI')
    quantum_module_XX = pauli_dict['XX'] * quantum_module(parameters, 'XX')
    quantum_module_YY = pauli_dict['YY'] * quantum_module(parameters, 'YY')
   
    # summing the measurement results
    classical_adder = quantum_module_II + quantum_module_IZ+ quantum_module_ZI + quantum_module_XX + quantum_module_YY
    return classical_adder


In [134]:
parameters_array = np.array([-np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.
a= {"maxiter" : 200}
vqe_result = minimize(vqe, parameters_array, method="COBYLA", tol=tol,options = a)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))
print("\nThe optimal parameter theta is : {} ".format(vqe_result.x))

The exact ground state energy is: -1.7491612220155843
The estimated ground state energy from VQE algorithm is: -1.7726937878

The optimal parameter theta is : [-5.6023758  2.3910904] 


# for H3

In [135]:
def hamiltonian_operator(a, b, c, d, e, f, g, h):
    """
    Creates a*III + b*IIZ + c*IZI + d*ZII + e*IXX + f*IYY + g*XXI + h*YYI pauli sum
    that will be our Hamiltonian operator.
   
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "III"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "IIZ"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "IZI"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "ZII"},
                   {"coeff": {"imag": 0.0, "real": e}, "label": "IXX"},
                   {"coeff": {"imag": 0.0, "real": f}, "label": "IYY"},
                   {"coeff": {"imag": 0.0, "real": g}, "label": "XXI"},
                   {"coeff": {"imag": 0.0, "real": h}, "label": "YYI"},
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [136]:
a, b, c, d, e, f, g, h = 15.531709, 0.218291, -6.125, -9.625, -2.143304, -2.143304, -3.913119, -3.913119
H = hamiltonian_operator(a, b, c, d, e, f, g, h)

In [137]:
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -2.0456722876770916


In [138]:
def vqe_circuit(parameters, measure):
    """Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'IIZ' stands for Z0 measurement,  'XXI' stands for X1X2 measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(3)
    c = ClassicalRegister(3)
    circuit = QuantumCircuit(q, c)
    
    circuit.x(0)
    circuit.ry(parameters[0],q[1])
    circuit.ry(parameters[1],q[2])
    circuit.cx(q[2], q[0])
    circuit.cx(q[0], q[1])
    circuit.barrier(q[0], q[1], q[2])
    circuit.ry(-parameters[0], q[1])
    circuit.cx(q[0], q[1])
    circuit.cx(q[1], q[0])
        
    
    # measurement
    if measure == 'IIZ':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[0], c[0])
    elif measure == 'IZI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[1], c[1])
    elif measure == 'ZII':
        circuit.barrier(q[0],q[1],q[2])
        circuit.measure(q[2], c[2])
    elif measure == 'IXX':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi, q[0])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'IYY':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi/2, q[0])
        circuit.u(np.pi/2,0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'XXI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi, q[1])
        circuit.u(np.pi/2, 0, np.pi, q[2])
        circuit.measure(q[1], c[1])
        circuit.measure(q[2], c[2])
    elif measure == 'YYI':
        circuit.barrier(q[0],q[1],q[2])
        circuit.u(np.pi/2, 0, np.pi/2, q[1])
        circuit.u(np.pi/2,0, np.pi/2, q[2])
        circuit.measure(q[1], c[1])
        circuit.measure(q[2], c[2])
    else:
        raise ValueError('Not valid input for measurement')
    
    return circuit


In [139]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'III':
        return 1
    elif measure == 'IIZ':
        circuit = vqe_circuit(parameters, 'IIZ')
    elif measure == 'IZI':
        circuit = vqe_circuit(parameters, 'IZI')
    elif measure == 'ZII':
        circuit = vqe_circuit(parameters, 'ZII')
    elif measure == 'IXX':
        circuit = vqe_circuit(parameters, 'IXX')
    elif measure == 'IYY':
        circuit = vqe_circuit(parameters, 'IYY')
    elif measure == 'XXI':
        circuit = vqe_circuit(parameters, 'XXI')
    elif measure == 'YYI':
        circuit = vqe_circuit(parameters, 'YYI')
    else:
        raise ValueError('Not valid input for measurement: input should be "III" or "IIZ" or "IZI" or "ZII" or "IXX" or "IYY" or "XXI" or "YYI"')
   
    shots = 10000
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = -1
        if (measure_result == '000') or (measure_result == '011')  or (measure_result == '101')  or (measure_result == '110'):
             sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [140]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {III: 15.531709, IIZ: 0.218291, IZI: -6.125, ZII: -9.625, IXX: -2.143304, IYY: -2.143304, XXI: -3.913119, YYI: -3.913119}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [141]:
def vqe(parameters):
       
    # quantum_modules
    quantum_module_III = pauli_dict['III'] * quantum_module(parameters, 'III')
    quantum_module_IIZ = pauli_dict['IIZ'] * quantum_module(parameters, 'IIZ')
    quantum_module_IZI = pauli_dict['IZI'] * quantum_module(parameters, 'IZI')
    quantum_module_ZII = pauli_dict['ZII'] * quantum_module(parameters, 'ZII')
    quantum_module_IXX = pauli_dict['IXX'] * quantum_module(parameters, 'IXX')
    quantum_module_IYY = pauli_dict['IYY'] * quantum_module(parameters, 'IYY')
    quantum_module_XXI = pauli_dict['XXI'] * quantum_module(parameters, 'XXI')
    quantum_module_YYI = pauli_dict['YYI'] * quantum_module(parameters, 'YYI')
   
    # summing the measurement results
    classical_adder = quantum_module_III + quantum_module_IIZ + quantum_module_IZI + quantum_module_ZII + quantum_module_IXX  + quantum_module_IYY + quantum_module_XXI + quantum_module_YYI
    return classical_adder


In [146]:
parameters_array = np.array([np.pi/2,np.pi/2])
tol = 1e-3 # tolerance for optimization precision.
a= {"maxiter" : 500}
vqe_result = minimize(vqe, parameters_array, method="COBYLA", tol=tol,options = a)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))
print("\nThe optimal parameter theta is : {} ".format(vqe_result.x))

The exact ground state energy is: -2.0456722876770916
The estimated ground state energy from VQE algorithm is: -1.9620622242000008

The optimal parameter theta is : [0.30666309 0.1797221 ] 


In [147]:
#trying to use qiskit optimizer
from qiskit.aqua.components.optimizers import COBYLA, SPSA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)
parameters_array = np.array([np.pi/2, np.pi/2])

vqe_results = optimizer.optimize(num_vars=2, objective_function=vqe, initial_point=parameters_array)
print(vqe_results[1])

-1.9717128774000026
